# Popularidade dos posts do Hacker News

Hacker News é um site criado pela Y Combinator, uma incubadora de startups, em que stories (posts) submetidos por usuários são votados e comentados, algo similar ao reddit. Hacker News é popular no círculo de pessoas que são interessadas em tecnologia ou trabalham em startups. Logo, os posts que chegam ao topo da listagem do site têm centenas de milhares de visitantes. 

Nesse projeto, um dataset dos posts da Hacker News disponibilizado no Kaggle será analisado. Como houve uma exclusão das submissões de usuários que não receberam nenhum comentário, o dataset foi reduzido de 300 mil linhas para 20 mil. 

O dataset tem as seguintes colunas:
- id: número único usado para identificar os posts
- title: o título dos posts
- url: O link dos posts
- num_points: o número de pontos que os posts adquiriram. O cálculo é feito com base no número total de upvotes reduzindo o - número total de downvotes
- num_comments: o número de comentários que foram feitos no post
- author: o nome do usuário de quem submeteu o post
- created_at: a data e horário em que um post foi submetido.

Para a análise, focaremos nos posts em que os títulos começam com Ask HN (questões específicas postadas pelas pessoas) e Show HN (projetos, produtos ou algo interessante postado para a comunidade do site). 

## Objetivo

A proposta é comparar os dois tipos de posts e determinar qual é mais popular, e se os posts criados em algum horário específico recebe, em média, mais comentários. 


## Parte Um: Explorando o dataset

O primeiro passo para o projeto, é explorar o dataset. Por isso, separo o cabeçalho. As informações importantes para a análise estão no título, número de comentários e, mais tarde, horário e data em que os posts foram criados.


In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
hn_header = hn[0]

hn_sample = hn[1:5]

print(hn_header)
print('\n')
print(hn_sample)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
hn = hn[1:] #removing the header from the data set

print(hn[1:5])

[['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Parte Dois: Analisando dois tipos de posts

Para começar a análise, os posts são separados em três listas diferentes, uma para 'Ask HN', outra para 'Show HN' e outra para qualquer outro tipo de postagem. Depois, a média de comentários para 'Ask HN' e 'Show HN' é calculada.


In [3]:
# .startwith() is a string method, it returns True or False

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lowercase = title.lower()
    if title_lowercase.startswith('ask hn'):
        ask_posts.append(row)
    elif title_lowercase.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)


print("O número de 'Ask HN' é:", len(ask_posts))
print("O número de 'Show HN' é:", len(show_posts))
print("O número de outros tipos de posts é:", len(other_posts))


O número de 'Ask HN' é: 1744
O número de 'Show HN' é: 1162
O número de outros tipos de posts é: 17194


In [4]:
total_ask_comments = 0
total_show_comments = 0

for row in ask_posts:
    comments = int(row[4])
    total_ask_comments += comments 

for row in show_posts:
    comments = int(row[4])
    total_show_comments += comments 

avg_ask_comments = total_ask_comments / len(ask_posts)
avg_show_comments = total_show_comments / len(show_posts)

print("A média de comentários em posts 'Ask HN' é:", avg_ask_comments)
print("A média de comentários em posts 'Show HN' é:", avg_show_comments)




A média de comentários em posts 'Ask HN' é: 14.038417431192661
A média de comentários em posts 'Show HN' é: 10.31669535283993


## Parte Três: Melhor horário para postagem

Como já foi notado que o tipo de postagem que recebe mais comentários, em média, é aquela que começa com 'Ask HN', só essa lista será utilizada nessa etapa da análise.

A primeira ação para identificar quais horários têm mais comentários é calcular o número de postagem em um determinado horário e o número de comentários. Assim, a média de comentários é calculada e um 'top 5' dos horários é criada. 


In [5]:
import datetime as dt

result_list = []
for row in ask_posts:
    result_list.append([row[6], int(row[4])])
    
counts_by_hour = {}
comments_by_hour = {}

date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = dt.datetime.strptime(row[0], date_format)
    hour = dt.datetime.strftime(date, "%H")
    comment = row[1]
    
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1 
        comments_by_hour[hour] += comment
    else:
        counts_by_hour[hour] = 1 
        comments_by_hour[hour] = comment

print(counts_by_hour)
print('\n')
print(comments_by_hour)
    



{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


In [6]:
avg_comments_hour = [] 

for hour in counts_by_hour:
    avg_comments_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])
    
print(avg_comments_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


In [10]:
swap_avg_by_hour = []

for row in avg_comments_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print('Lista em ordem:', sorted_swap)


Lista em ordem: [[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [11]:
print("Top 5 Horários para comentários:")
for avg, hr in sorted_swap[0:5]:
   print('{}: {avg:,.2f} average comments per post'. format(dt.datetime.strptime(hr,'%H').strftime("%H"), avg = avg))



Top 5 Horários para comentários:
15: 38.59 average comments per post
02: 23.81 average comments per post
20: 21.52 average comments per post
16: 16.80 average comments per post
21: 16.01 average comments per post


## Conclusão

Nota-se que os posts que têm questões para a comunidade do site Hacker News recebem mais comentários do que aqueles que fazem sugestões ou apresentam projetos. A média para os posts classificados como 'Ask HN' é de 14.03 e para os 'Show HN' é de 10.31. 

Com essa observação, apenas as postagem 'Ask HN' são selecionadas para a segunda etapa da análise. Com esses dados, observa-se que os horários com melhores médias de comentários são: 15h, 02h, 20h, 16h e 21h. 

